In [28]:
import pandas as pd
df = pd.read_csv("./downloads/reviews100.csv")
df['comment']

0     \n\n\n\n\n\n\n\n\n\n  \n  \n    \n  I purchase...
1     \n\n\n\n\n\n\n\n\n\n  \n  \n    \n  This is a ...
2     \n\n\n\n\n\n\n\n\n\n  \n  \n    \n   PROS: I w...
3     \n\n\n\n\n\n\n\n\n\n  \n  \n    \n  I purchase...
4     \n\n\n\n\n\n\n\n\n\n  \n  \n    \n  I purchase...
                            ...                        
95    \n\n\n\n\n\n\n\n\n\n  \n  \n    \n   This shre...
96    \n\n\n\n\n\n\n\n\n\n  \n  \n    \n  Over the y...
97    \n\n\n\n\n\n\n\n\n\n  \n  \n    \n  ​I purchas...
98    \n\n\n\n\n\n\n\n\n\n  \n  \n    \n  I shred ju...
99    \n\n\n\n\n\n\n\n\n\n  \n  \n    \n  Returned a...
Name: comment, Length: 100, dtype: object

In [41]:
df['comment'] = df['comment'].apply(lambda x: x.strip())
df.sort_values(['stars'], ascending=False).iloc[0:40]

,stars,comment
77,5.0 out of 5 stars,"Over the years I've used a full-size shredder,..."
24,5.0 out of 5 stars,I am not someone who uses a paper shredder eve...
26,5.0 out of 5 stars,"Over the years I've used a full-size shredder,..."
56,5.0 out of 5 stars,This shredder works as advertised. In the atta...
76,5.0 out of 5 stars,This shredder works as advertised. In the atta...
75,5.0 out of 5 stars,I am not someone who uses a paper shredder eve...
31,5.0 out of 5 stars,I shred junk mail like it's a part time job. S...
36,5.0 out of 5 stars,I shred junk mail like it's a part time job. S...
68,5.0 out of 5 stars,I shred junk mail like it's a part time job. S...
39,5.0 out of 5 stars,I shred junk mail like it's a part time job. S...


,stars,comment
0,4.0 out of 5 stars,I purchased this bad boy for my wife’s home of...
1,4.0 out of 5 stars,This is a quite nice machine. Very basic yet b...
2,4.0 out of 5 stars,PROS: I want to cover first the high-quality a...
3,4.0 out of 5 stars,I purchased this bad boy for my wife’s home of...
4,4.0 out of 5 stars,I purchased this bad boy for my wife’s home of...
...,...,...
95,5.0 out of 5 stars,This shredder works as advertised. In the atta...
96,5.0 out of 5 stars,"Over the years I've used a full-size shredder,..."
97,4.0 out of 5 stars,​I purchased this shredder 3 months ago. Love ...
98,5.0 out of 5 stars,I shred junk mail like it's a part time job. S...
